
In this section, we demonstrate the generation of single-cell–level regulatory graphs required for running scReGAT on unmatched multi-omics datasets.

As an illustrative example, we employ human cerebral cortex data, originally reported by Mulqueen et al. (scATAC-seq) and Hodge et al. (scRNA-seq).

In [1]:
import os
import sys
import random
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
from scregat.data_process import prepare_model_input, sum_counts, plot_edge, ATACGraphDataset
import scanpy as sc
import itertools
import torch

/opt/conda/envs/scReGAT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/scReGAT/lib/python3.10/site-packages/anndata/_metadata.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
import scregat
import os

print(scregat.data_process.__file__)              # 显示模块文件的完整路径
print(os.path.dirname(scregat.__file__))  # 显示所在目录


/root/scReGAT/scregat/data_process.py
/root/scReGAT/scregat


In [3]:
ATAC_h5ad_file = "../../HumanBrain_ATAC.h5ad"
RNA_h5ad_file = "../../HumanBrain_RNA.h5ad"


# - It is important to note that both **adata_rna** and **adata_atac** must contain a `celltype` column in their `obs`.  
# - The cell types present in **adata_atac** must also be represented in the `obs` of **adata_rna**.  
# - The `celltype` entries in **adata_rna** are allowed to include a larger number of categories.  


In [4]:
adata_atac = sc.read_h5ad(ATAC_h5ad_file)
adata_rna = sc.read_h5ad(RNA_h5ad_file)


In [5]:
adata_atac

AnnData object with n_obs × n_vars = 2174 × 292156
    obs: 'celltype', 'celltype_rna'
    var: 'peak'

In [6]:
adata_atac.obs['celltype_rna'] == adata_atac.obs['celltype']
adata_atac.obs

,celltype,celltype_rna
GAAGCAGCTCTGACGACCGCGGTT,inhibitory_neuron,inhibitory_neuron
TCCATACCAATGATGCGGCATTCT,oligodendrocytes,oligodendrocytes
ATTGAGGAACCGGAAGACACTAAG,oligodendrocytes,oligodendrocytes
ACGCGACGTCTGACGAGCCACAGG,oligodendrocytes,oligodendrocytes
ATTGAGGAAGTTACGCTCCAACGC,oligodendrocytes,oligodendrocytes
...,...,...
GAAGAGTAGCCAAGGCGTGAATAT,oligodendrocytes,oligodendrocytes
GAAGCAGCTTAACTCAGGTACCTT,oligodendrocytes,oligodendrocytes
GGTTAGTTAGTTACGCTTGGACTC,polydendrocytes,polydendrocytes
GAAGAGTAATGGAGCTCTTGGTAT,oligodendrocytes,oligodendrocytes


In [7]:
adata_atac.var

,peak
chr10-10113-10613,chr10:10113-10613
chr10-74022-74522,chr10:74022-74522
chr10-76384-76884,chr10:76384-76884
chr10-129575-130075,chr10:129575-130075
chr10-134348-135024,chr10:134348-135024
...,...
chrX-155148867-155149367,chrX:155148867-155149367
chrX-155215853-155217198,chrX:155215853-155217198
chrX-155263318-155263818,chrX:155263318-155263818
chrX-155263973-155264786,chrX:155263973-155264786


In [8]:
adata_rna

AnnData object with n_obs × n_vars = 15603 × 50281
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cluster', 'class', 'brain_subregion', 'donor', 'sex', 'facs_sort_criteria', 'seq_batch', 'total_reads', 'percent_reads_unique', 'celltype'

In [9]:
adata_rna.obs

,orig.ident,nCount_RNA,nFeature_RNA,cluster,class,brain_subregion,donor,sex,facs_sort_criteria,seq_batch,total_reads,percent_reads_unique,celltype
F1S4_160106_001_B01,0,1945567.0,8635,Inh L4-6 SST B3GAT2,GABAergic,L5,H200.1030,M,NeuN-positive,R8S4-160411-H,2572946,85.573696,inhibitory_neuron
F1S4_160106_001_C01,0,2076398.0,11697,Exc L5-6 RORB TTC12,Glutamatergic,L5,H200.1030,M,NeuN-positive,R8S4-160411-H,2755839,87.619777,excitatory_neuron
F1S4_160106_001_E01,0,1984845.0,12138,Exc L5-6 FEZF2 ABO,Glutamatergic,L5,H200.1030,M,NeuN-positive,R8S4-160411-H,2701064,86.448562,excitatory_neuron
F1S4_160106_001_G01,0,1991032.0,12191,Exc L5-6 FEZF2 EFTUD1P1,Glutamatergic,L5,H200.1030,M,NeuN-positive,R8S4-160411-H,2759117,84.497105,excitatory_neuron
F1S4_160106_001_H01,0,2189892.0,10535,Exc L3-5 RORB ESR1,Glutamatergic,L5,H200.1030,M,NeuN-positive,R8S4-160411-H,2930410,87.014752,excitatory_neuron
...,...,...,...,...,...,...,...,...,...,...,...,...,...
F2S4_170405_060_B01,1,2315521.0,10853,Exc L5-6 FEZF2 ABO,Glutamatergic,L5,H16.06.008,F,"SATB2-pos, NeuN-pos",R8S4-170505,3015621,89.829491,excitatory_neuron
F2S4_170405_060_C01,1,2376631.0,10539,Exc L5-6 THEMIS FGF10,Glutamatergic,L5,H16.06.008,F,"SATB2-pos, NeuN-pos",R8S4-170505,3121536,89.871717,excitatory_neuron
F2S4_170405_060_E01,1,1761501.0,9017,Exc L4-6 FEZF2 IL26,Glutamatergic,L5,H16.06.008,F,"SATB2-neg, NeuN-pos",R8S4-170505,2771628,73.988717,excitatory_neuron
F2S4_170405_060_F01,1,1736137.0,4953,Inh L4-6 SST B3GAT2,GABAergic,L5,H16.06.008,F,"SATB2-neg, NeuN-pos",R8S4-170505,2307182,86.352789,inhibitory_neuron


In [10]:
adata_rna.var

""
3.8-1.2
3.8-1.3
3.8-1.4
3.8-1.5
5-HT3C2
...
ZYX
ZZEF1
ZZZ3
bA255A11.4


In [11]:
adata_atac.obs.celltype.unique()

['inhibitory_neuron', 'oligodendrocytes', 'microglia', 'excitatory_neuron', 'astrocyte', 'polydendrocytes']
Categories (6, object): ['astrocyte', 'excitatory_neuron', 'inhibitory_neuron', 'microglia', 'oligodendrocytes', 'polydendrocytes']

In [12]:
adata_rna.obs.celltype.unique()

['inhibitory_neuron', 'excitatory_neuron', 'oligodendrocytes', 'OPC', 'astrocyte', 'microglia', 'polydendrocytes']
Categories (7, object): ['OPC', 'astrocyte', 'excitatory_neuron', 'inhibitory_neuron', 'microglia', 'oligodendrocytes', 'polydendrocytes']

In [13]:
adata_rna.obs['celltype'] = adata_rna.obs['celltype'].astype('object')
df_rna = sum_counts(adata_rna,by = 'celltype', marker_gene_num=300)

**finished identifying marker genes by COSG**


In [14]:
df_rna

,CYTH4,ARHGAP26,CPNE4,HIF3A,RFX4,TENM1,EVI2A,MIR124-2HG,LOC101928908,PLEKHA7,...,ZNF462,ABHD2,LINC01098,PROX1,BDP1,CBR1,CDC42EP1,LAMC3,LSP1,MAP6D1
celltype,,,,,,,,,,,,,,,,,,,,,
OPC,0.080587,39.386703,11.987419,73.010239,18.216758,387.411222,14.155792,34.142796,0.158801,12.470241,...,307.400824,342.492397,4.172463,17.616328,98.311920,30.703984,2.380326,2.398969,0.230454,5.864957
astrocyte,0.135728,406.617399,15.526756,347.018572,118.715937,10.425881,16.306267,50.932324,2.132234,204.198082,...,173.438055,152.326999,15.778694,0.228048,159.084881,43.524882,1.192625,2.013769,0.187578,6.515867
excitatory_neuron,26.367287,18265.602317,10978.021716,88.317756,141.790249,5130.059192,611.492859,15172.219813,60.042778,344.440834,...,1457.325977,4120.093278,82.935006,35.056608,15222.575657,529.984062,3.510019,116.042034,48.892340,639.356139
inhibitory_neuron,7.590384,4546.410687,474.333750,163.549419,52.962522,5842.873550,216.584376,4103.218805,14.423904,467.100721,...,1601.356152,2179.067687,41.364060,1389.439627,4428.039174,228.222706,4.863837,211.576218,12.411594,151.607610
microglia,32.846013,124.543262,6.068610,1.557616,0.029130,1.413688,26.661758,0.060154,0.825634,2.862510,...,0.124285,8.075438,0.895430,0.701079,40.106962,1.721333,0.949158,0.250513,0.073228,0.850794
oligodendrocytes,0.144513,30.997706,14.853391,7.211505,2.870729,18.854445,345.282072,0.801757,1.232896,40.729483,...,95.315052,99.274318,83.591684,115.363503,145.625163,218.330363,39.590520,11.277164,1.194011,130.431312
polydendrocytes,0.007974,8.133928,0.161308,6.306033,0.893003,0.052720,0.007974,0.000000,2.314232,1.183778,...,1.872045,1.555839,0.000000,0.000000,8.053840,3.155592,0.000000,4.629706,1.855250,0.000000


In [15]:
1

1

In [16]:
import scregat
import os
# This step adds tissue-specific Hi-C regulatory relationships.
# The user needs to provide a set of files. For example,
#in 
base_dir = '../data/'
os.listdir(base_dir)
# we include a file called PO_brain.txt,
# which contains brain tissue-specific Hi-C links.


['TF_Gene_tissue_Brain.csv',
 'PO.txt',
 'PO_brain.txt',
 'readme.md',
 'hg38.chrom.sizes',
 'PP.txt',
 'celltype_specific_cRE_interactions',
 'TF_Gene_tissue_cutoff1.csv',
 'all_tissue_SNP_Gene.txt',
 'genes.protein.tss.tsv',
 'trrust_rawdata.human.tsv',
 'PP_brain.txt',
 'peaks_process',
 'model_init.pth',
 'processed_files']

In [17]:
dataset_obj = prepare_model_input(
    # [Core Data] Single-cell ATAC-seq AnnData object.
    # Requirement: .X must be Peak-by-Cell matrix; .obs must contain cell type annotations.
    adata_atac = adata_atac,
    
    # [Output Path] Root directory for storing intermediate processed files.
    # The script will create a 'processed_files' folder here (e.g., sorted bed files).
    path_data_root = '../',
    
    # [File Reference] String path to the original ATAC file (used for naming/logging).
    file_atac = ATAC_h5ad_file, 
    
    # [Core Data] RNA expression matrix aggregated by cell type (Pseudo-bulk).
    # Format: Pandas DataFrame. 
    # Index (Rows): Cell type names (must match 'celltype_rna' in adata_atac.obs).
    # Columns: Gene Symbols (e.g., 'TP53').
    df_rna_celltype = df_rna,
    
    # [Prior Knowledge] Path to eQTL (expression Quantitative Trait Loci) data.
    # Used to link SNPs/non-coding regions to target genes.
    path_eqtl = '../data/all_tissue_SNP_Gene.txt',
    
    # [Prior Knowledge] Suffix for Hi-C interaction files to specify tissue context.
    # e.g., if set to "_brain", the code looks for 'PP_brain.txt' and 'PO_brain.txt'.
    # PP = Promoter-Promoter, PO = Promoter-Other (Enhancer).
    Hi_C_file_suffix = "_" + "brain",  
    
    # [Preprocessing] Whether to convert genomic coordinates from hg19 to hg38.
    # Set True if input ATAC peaks are hg19 (requires LiftOver tool); False if already hg38.
    hg19tohg38 = False,
    
    # [QC Filter] Peak filtering threshold.
    # Peaks must be accessible in at least 1% (0.01) of cells to be retained.
    min_percent = 0.01,
    
    # [Prior Knowledge] Whether to use an extended TF (Transcription Factor) database.
    # False: Uses TRRUST only (Curated, high-confidence).
    # True: Uses TRRUST + CHEA3/ChIP-seq aggregated data.
    use_additional_tf = True,
    
    # [Prior Knowledge] Reliability threshold for the extended TF database.
    # Only applies if use_additional_tf=True.
    # Integer indicating in how many tissues/datasets the TF-Gene link must appear to be kept.
    # 10 indicates a high-confidence, conserved regulatory relationship.
    tissue_cuttof = 10
)

only dataset_obj ...
processing Hi-C ...


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


processing TF ...
additional TF...
total candidate tf-gene:  28054


In [18]:
dataset_obj.list_graph[0]

Data(x=[61528, 1], edge_index=[2, 69180], y=[1], edge_tf=[155, 2], y_exp=[1348], cell='TCCATACCAATGATGCGGCATTCT')

In [21]:
file_atac_test = os.path.join('./', 'dataset_atac_kRG_MFG.pkl')
with open(file_atac_test, 'wb') as w_pkl:
    str_pkl = pickle.dumps(dataset_obj)
    w_pkl.write(str_pkl)

### Add Tissue-specific TF-gene  (from CHEA3)

In [22]:
df = pd.read_csv("./scReGAT/data/TF_Gene_tissue_Brain.csv", index_col=0)
df.columns = ['TF', 'TargetGene', 'tissue_count']
df_tf = df

In [23]:
df_tf.head()

,TF,TargetGene,tissue_count
0,TFAP2A,KDM4B,1
1,TFAP2A,PLXNA1,1
2,TFAP2A,LYPD6B,1
3,TFAP2A,ESRP1,1
4,TFAP2A,MAB21L1,1


In [24]:
# Extract all gene names from the RNA dataframe columns
gene_list = list(dataset_obj.df_rna.columns)

# Convert gene list to a set for faster membership checks
set_gene = set(gene_list)

# Filter TF dataframe to keep only rows where both TF and TargetGene are in the gene set
tf_base_filtered = df_tf[df_tf['TF'].isin(set_gene) & df_tf['TargetGene'].isin(set_gene)]

# Generate all possible gene pairs (Cartesian product of gene set with itself)
connections = [pair for pair in itertools.product(set_gene, set_gene)]
gene_pair_base = connections

# Collect unique TFs and TargetGenes from the filtered dataframe
tf_map_gene = set(tf_base_filtered['TF'].unique())
target_map_gene = set(tf_base_filtered['TargetGene'].unique())

# Create tuples of (TF, TargetGene) from the filtered dataframe
tf_base_tuples = set(zip(tf_base_filtered['TF'], tf_base_filtered['TargetGene']))

# Intersect with all possible gene pairs to keep only valid TF-target pairs
map_pair = tf_base_tuples.intersection(gene_pair_base)

# Convert the set of valid pairs into a list
map_pair_list = list(map_pair)

# Create a new dataframe from the valid TF-target pairs
df_tf_new = pd.DataFrame(map_pair_list, columns=['TF', 'TargetGene'])

# Concatenate the new TF-target dataframe with the existing one in dataset_obj
df_tf_all = pd.concat([df_tf_new, dataset_obj.df_tf])

# Remove duplicate rows to ensure uniqueness
df_tf_all = df_tf_all.drop_duplicates()

# Update dataset_obj with the merged TF-target dataframe
dataset_obj.df_tf = df_tf_all


In [25]:
# Create a dictionary to store the index of each element in dataset_atac.array_peak
peak_index_dict = {peak: idx for idx, peak in enumerate(dataset_obj.array_peak)}

# Initialize lists to store indices
index_1 = []
index_2 = []

# Iterate over 'TF' and 'TargetGene' columns in dataset_atac.df_tf
for k1, k2 in zip(dataset_obj.df_tf['TF'].values, dataset_obj.df_tf['TargetGene'].values):
    # Use the dictionary to quickly retrieve indices
    index_1.append(peak_index_dict[k1])
    index_2.append(peak_index_dict[k2])

# Stack the two index lists column-wise and convert to a PyTorch tensor
tf_edge_vec = torch.tensor(np.vstack([index_1, index_2]).T)

# Assign the TF edge tensor to the edge_tf attribute of each graph in the list
for t in dataset_obj.list_graph:
    t.edge_tf = tf_edge_vec


In [26]:
dataset_obj.list_graph[0]

Data(x=[61528, 1], edge_index=[2, 69180], y=[1], edge_tf=[4036, 2], y_exp=[1348], cell='TCCATACCAATGATGCGGCATTCT')

In [27]:
file_atac_test = os.path.join('./', 'dataset_atac_kRG_MFG.pkl')
with open(file_atac_test, 'wb') as w_pkl:
    str_pkl = pickle.dumps(dataset_obj)
    w_pkl.write(str_pkl)


## Training the Model and Obtaining Regulatory Scores

You can train the model and generate regulatory scores directly from the command line.  
For example, we used the following command to produce a sample dataset and a model weight file:

```bash
python ./scReGAT/run_scregat_cli.py \
  --input_file ./dataset_atac_kRG_MFG.pkl \
  --output_file ./RS_score.h5ad \
  --save_model_path ./scReGAT/data/model_init.pth \
  --gpu 2
```

---

## Argument Description

```python
def parse_args():
    parser = argparse.ArgumentParser(description="Run scReGAT Model Training and Inference")

    # --- I/O parameters ---
    parser.add_argument('--input_file', type=str, required=True, 
                        help='Path to the input ATAC pickle file (e.g., dataset_atac_core_MFG.pkl)')
    parser.add_argument('--output_file', type=str, required=True, 
                        help='Path to save the output AnnData file (e.g., result.h5ad)')

    # --- Single-cell expression integration parameters ---
    parser.add_argument('--use_sc_exp', action='store_true',
                        help='Enable integration of single-cell RNA expression data into graph node features')
    parser.add_argument('--rna_file', type=str, default=None,
                        help='Path to the RNA .h5ad file; required if --use_sc_exp is set')

    # --- Model saving and loading ---
    parser.add_argument('--save_model_path', type=str, default=None, 
                        help='Optional path to save trained model parameters')
    parser.add_argument('--load_model_path', type=str, default=None, 
                        help='Optional path to load pre-trained model parameters')

    # --- Training control ---
    parser.add_argument('--skip_train', action='store_true',
                        help='Skip the training phase and run inference directly')
    parser.add_argument('--seed', type=int, default=1233, 
                        help='Random seed (default: 1233)')
    parser.add_argument('--epochs', type=int, default=4, 
                        help='Number of training epochs (default: 4)')
    parser.add_argument('--lr', type=float, default=1e-4, 
                        help='Learning rate (default: 1e-4)')
    parser.add_argument('--batch_size', type=int, default=15, 
                        help='Training batch size (default: 15)')
    parser.add_argument('--sparse_loss_weight', type=float, default=0.1, 
                        help='Weight for sparse loss (default: 0.1)')

    # --- Testing / inference parameters ---
    parser.add_argument('--test_batch_size', type=int, default=20, 
                        help='Batch size for inference (default: 20)')
    parser.add_argument('--test_ratio', type=float, default=0.5, 
                        help='Ratio of cells to use for testing (default: 0.5)')

    # --- Hardware parameters ---
    parser.add_argument('--gpu', type=int, default=1, 
                        help='GPU ID to use; set -1 to use CPU (default: 1)')

    return parser.parse_args()
```

---

## Recommendation

We recommend users initialize training on new datasets using the model weight file:

```
./scReGAT/data/model_init.pth
```

This provides a stable starting point for training and helps ensure stability across experiments.

---
